Code for creating the animations in the `plots/paths/animated` folder.

In [1]:
%matplotlib inline
%pylab inline
import xarray as xr
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import matplotlib.colors, matplotlib.collections
import cycler

from matplotlib import animation, rc
from IPython.display import HTML
rc('animation', html='html5')

Populating the interactive namespace from numpy and matplotlib


In [2]:
dataset = '2014'
data = xr.open_dataset(f'../xarray/{dataset}.nc',engine='h5netcdf')

In [3]:
d = data.isel(rat=0,session=0)

In [4]:
def render_path(ds,fps=10,speed=20,window=5,limit=float('inf'),data_fps=25):
    
    fig, ax = plt.subplots()

    #cm = plt.get_cmap('Greys')
    #ax.set_prop_cycle(cycler.cycler('color',[cm(1.*i/(window-1)) for i in range(window*data_fps-1)]))
    ax.set_prop_cycle(cycler.cycler('alpha',[i/(window-1) for i in range(window-1)]))

    ax.set_xlim((-100, 100))
    ax.set_ylim((-100, 100))
    ax.set(aspect='equal')

    circles = [((46.5, -42.5),7),((4.5, 44),7),((-69, -69),6),((-66.5, 73.5),6)]
    for circle,radius in circles:
        ax.add_artist(plt.Circle(circle, radius, color='k', fill=False, lw=1))

    edges = ([(-61.5,78),(-30,80.125),(0,80.75),(30,80.5),(60,78.5),
              (81.5,76),(84.75,44),(84.75,10.5),(82.5,-28.75),
              (76,-74),(43.75,-77.25),(7.75,-77.25),(-27.25,-75.25),
              (-62.75,-70.25)],
             [(-70.25,-62.25),(-73.75,-30),(-74.5,0.25),(-73.5,38.25),(-69.75,68)])
    for edge in edges:
        plt.plot(*zip(*edge),lw=1,color='k',alpha=1)

    text = plt.text(-90, -90, '', fontsize=12)
    
    full_line, = ax.plot([], [], lw=3, c='k', alpha=0.1)
    lines = [ax.plot([], [], lw=3, c='k')[0] for _ in range(window-1)]

    def init():
        for line in lines:
            line.set_data([], [])
        return lines

    data_per_sec = data_fps*speed
    data_per_frame = data_per_sec//fps

    def animate(i):
        time = d.time[i*data_per_frame].data
        seconds = time / np.timedelta64(1, 's')
        timestr = ':'.join(str(datetime.timedelta(0,seconds)).split(':')[1:])
        text.set_text(timestr)

        s = i*data_per_frame-window*data_fps
        e = i*data_per_frame
        x = ds.x[s:e]
        y = ds.y[s:e]

        full_line.set_data(ds.x[:e],ds.y[:e])
        for i in range(window-1):
            s = i*data_fps
            e = (i+2)*data_fps
            lines[i].set_data(x[s:e],y[s:e])

        return lines
    
    an = animation.FuncAnimation(fig, animate, init_func=init, frames=min(len(d.time),limit)//data_per_frame, interval=1000/fps, blit=True)
    plt.close()
    return an

In [5]:
render_path(d,limit=1500)

In [7]:
for rat in data.rat:
    for session in data.session:
        if rat.data not in ['109','110','112','114','111'] or rat.data=='111' and session.data in ['injection7','injection8','injection9','injection10']:
            d = data.sel(rat=rat,session=session)
            an = render_path(d)
            an.save(f'../plots/paths/animated/{dataset}_{rat.data}_{session.data}.mp4', writer='ffmpeg')

KeyboardInterrupt: 